# Урок 6. Типы медицинских данных
Домашнее задание: John Snow Labs предоставляют простой и унифицированный API Python для обработки текстовых данных с помощью NLP на профессиональном уровне.

Изучите одно из предлагаемых решений процессинга клинических данных по поиску именованных сущностей (NER).
https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.Clinical_Named_Entity_Recognition_Model.ipynb

Протестируйте распознавание моделью терминов на тексте из какой-либо статьи раздела Case Reports в PubMed NCBI.

In [30]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql.functions import explode, col

# Создаем компоненты
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

tokenizer = Tokenizer() \
    .setInputCols(["documents"]) \
    .setOutputCol("tokens")

# Загружаем модель NER
ner_model = BertForTokenClassification.pretrained("bert_finetuned_ner_clinical_plncmm_large_22", "en") \
    .setInputCols(["documents", "tokens"]) \
    .setOutputCol("ner")

# Создаем pipeline
pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    ner_model
])

# Текст статьи или фрагмента
text = """
A 35-year-old woman diagnosed with fibrodysplasia ossificans progressiva (FOP) presented with progressive heterotopic ossification. The condition is characterized by abnormal bone formation outside the skeleton, often following trauma. Treatment options remain limited.
"""

# Создаем DataFrame
data = spark.createDataFrame([[text]]).toDF("text")

# Обучение и применение модели
pipeline_model = pipeline.fit(data)
result = pipeline_model.transform(data)

# Обработка результатов
# Распаковать массив сущностей
entities = result.select(explode(col("ner")).alias("entity")) \
    .select(
        col("entity.result").alias("entity_text"),
        col("entity.metadata").getItem("entity").alias("entity_type")
    )

entities.show(truncate=False)

bert_finetuned_ner_clinical_plncmm_large_22 download started this may take some time.
Approximate size to download 390.7 MB
[OK!]
+-----------+-----------+
|entity_text|entity_type|
+-----------+-----------+
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|B-Disease  |NULL       |
|I-Disease  |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
+-----------+-----------+
only showing top 20 rows

